# Genetic Algorithm for Timetabling Problem

## 0 - Configurations


## 0.1 - Importing Libraries

In [2439]:
from dataclasses import dataclass, field
from typing import List, Dict, Set, Tuple, Any, Optional
import time
import random
import json

## 0.2 - Environmental Variables

In [2440]:
MOCK_DATA_FOLDER_PATH = "../../../data/mock/"
PROFESSORS_FILE_PATH = MOCK_DATA_FOLDER_PATH + "teachers.json"
SUBJECTS_FILE_PATH = MOCK_DATA_FOLDER_PATH + "subjects.json"
CLASSROOMS_FILE_PATH = MOCK_DATA_FOLDER_PATH + "classrooms.json"
CLASSGROUPS_FILE_PATH = MOCK_DATA_FOLDER_PATH + "classgroup.json"
DAYS_FILE_PATH = MOCK_DATA_FOLDER_PATH + "days.json"

## 0.3 - Dataclasses

In [2441]:
@dataclass
class ClassGroup:
    id: int
    name: str
    students: int

    def __init__(self, class_group_id: int, class_group_data: dict) -> None:
        self.id = class_group_id
        self.name = class_group_data["name"]
        self.students = class_group_data["studentsAmount"]

In [2442]:
@dataclass
class Subject:
    id: int
    name: str
    professors: List[str]
    course_load: int

    def __init__(self, subject_name: str, subject_data: dict) -> None:
        self.id = subject_data["id"]
        self.name = subject_name
        self.professors = subject_data["professors"]
        self.course_load = subject_data["course_load"]

In [2443]:
@dataclass
class Professor:
    id: int
    name: str
    time_available: Dict[str, List[str]]
    subjects_taughtble_id: List[int] = field(default_factory=list)

    def __init__(self, professor_name: str, professor_data: dict) -> None:
        self.id = professor_data["id"]
        self.name = professor_name
        self.subjects_taughtble_id = professor_data.get("subjects_taughtble_id", [])
        self.time_available = professor_data.get("time_available", {})

    @property
    def availability(self) -> List[Set[str]]:
        availability = []
        for day, times in self.time_available.items():
            for time in times:
                availability.append((day, time))
        return availability

In [2444]:
@dataclass
class Classroom:
    id: int
    name: str
    time_available: Dict[str, List[int]]
    capacity: int

    def __init__(self, classroom_name: str, classroom_data: dict) -> None:
        self.id = classroom_data["id"]
        self.name = classroom_name
        self.time_available = classroom_data["time_available"]
        self.capacity = classroom_data["capacity"]

    @property
    def availability(self) -> List[Set[str]]:
        availability = []
        for day, times in self.time_available.items():
            for time in times:
                availability.append((day, time))
        return availability

In [2445]:
@dataclass
class Gene:
    classgroup_id: int
    classgroup_students: int
    subject_id: int
    professor_id: int
    professor_day: str
    professor_time: int
    classroom_capacity: int

    def __init__(self, classgroup_id: int, classgroup_students: int, subject_id: int, professor_id: int, professor_day: str, professor_time: int, classroom_capacity: int) -> None:
        self.classgroup_id = classgroup_id
        self.classgroup_students = classgroup_students
        self.subject_id = subject_id
        self.professor_id = professor_id
        self.professor_day = professor_day
        self.professor_time = professor_time
        self.classroom_capacity = classroom_capacity

@dataclass
class Chromosome:
    genes: List[Gene]

    def __init__(self) -> None:
        self.genes = []

    def add_gene(self, gene: Gene) -> None:
        self.genes.append(gene)
        print(f"Gene added: {gene}")

@dataclass
class Population:
    chromosomes: List[Tuple[Chromosome, float]] = field(default_factory=list)

    def __init__(self) -> None:
        self.chromosomes = []

    def __iter__(self):
        return (chromosome for chromosome, _ in self.chromosomes)

    def __getitem__(self, index: int) -> Tuple[Chromosome, float]:
        return self.chromosomes[index]

    def __len__(self):
        return len(self.chromosomes)

    def add_chromosome(self, chromosome: Chromosome, fitness: float = 0) -> None:
        self.chromosomes.append((chromosome, fitness))

    def generate_population_from_data(self, population_size: int, subjects: Dict[str, Subject], professors: Dict[str, Professor], classrooms: Dict[str, Classroom], classgroups: Dict[str, ClassGroup]) -> None:
        for _ in range(population_size):
            chromosome = Chromosome()
            for classgroup_name, classgroup in classgroups.items(): # Para cada turma com x alunos
                for subject_name, subject in subjects.items(): # Para cada disciplina
                    for professor_name in subject.professors: # Para cada professor que habilitado para dar a disciplina
                        professor = professors[professor_id]
                        for classroom_name, classroom in classrooms.items(): # Para cada sala
                            for hous in range(subject.course_load // 36): # Para cada aula na semana (1 ou 2 aulas por semana a depender da carga horária da disciplina)
                                day = random.choice(list(professor.time_available.keys())) # Dia aleatório dos dias disponíveis do professor
                                time = random.choice(professor.time_available[day]) # Horário aleatório dos horários disponíveis do professor
                                gene = Gene(professor.name, subject.name, day, time, classgroup, classroom)
                            chromosome.add_gene(gene)
            self.add_chromosome(chromosome)

In [2446]:
@dataclass
class EvaluationMetrics:
    iterations: int = 0
    best_iteration: int = 0
    avg_conflicts_history: List[int] = field(default_factory=list)
    avg_elite_fitness_history: List[float] = field(default_factory=list)
    time_to_evaluate: List[float] = field(default_factory=list)
    best_timetable: Any = None
    best_conflicts: int = 0
    best_elite_fitness: Optional[float] = None
    time_to_converge: float = 0

## 0.3 - Custom Classes

In [2447]:
class MetricsEvaluator:
    def __init__(self, model: str) -> None:
        self.model = model
        self.metrics = {
            "time_to_converge": 0,
            "best_fitness": 0,
            "best_chromosome": None,
            "best_chromosome": None,
            "best_generation": None,
            "best_conflicts": None
        }
        self.iteration_times = []

    def __repr__(self) -> str:
        return f"MetricsEvaluator({self.model})"

    def __str__(self) -> str:
        return f"MetricsEvaluator of Model: {self.model} with Metrics: {self.metrics}"

    def start_timer(self):
        self.start_time = time.time()

    def start_iteration_timer(self):
        self.iteration_start_time = time.time()

    def stop_iteration_timer(self):
        iteration_duration = time.time() - self.iteration_start_time
        self.iteration_times.append(iteration_duration)

    def report(self):
        print(f"Total time: {self.total_time} seconds")
        for i, duration in enumerate(self.iteration_times):
            # print(f"Iteration {i + 1}: {duration} seconds")
            pass

    def stop_timer(self):
        self.total_time = time.time() - self.start_time
        self.metrics["time_to_converge"] = self.total_time // 60

    def update_metrics(self, best_chromosome: Chromosome, best_fitness: float):
        self.metrics["best_fitness"] = best_fitness
        self.metrics["best_chromosome"] = best_chromosome


    def update_best_metric(self, generation, chromosome, conflicts, fitness):
        self.metrics["best_generation"] = generation
        self.metrics["best_chromosome"] = chromosome
        self.metrics["best_conflicts"] = conflicts
        self.metrics["best_fitness"] = fitness
        print(f"Best metrics updated: Generation {generation}, Fitness {fitness}, Conflicts {conflicts}")

## 0.4 - Custom Functions

In [2448]:
def load_data(file_path, data_class) -> dict:
    with open(file_path, 'r') as file:
        data = json.load(file)
        for key, value in data.items():
            data[key] = data_class(key, value)
        return data

In [2449]:
def calculate_quantity_of_classes(subjects: List[Dict[str, Any]], classrooms: Dict[str, Dict[str, Any]]) -> float:
    quantity_of_classes: float = 0
    for subject in subjects:
        course_load = subject['course_load']
        quantity_of_classes += course_load // 1.5
    quantity_of_classes = quantity_of_classes * len(classrooms)
    return quantity_of_classes

In [2450]:
def generate_initial_population(population_size, subjects, professors, classrooms, classgroups):
    population = Population()
    for _ in range(population_size):
        chromosome = Chromosome()
        for subject in subjects:
            subject_id = subject['id']
            for classgroup in classgroups:
                classgroup_id = classgroup['id']
                for professor_id in subject['professors']:
                    professor_key = str(professor_id)
                    if professor_key in professors:
                        professor = professors[professor_key]
                        for classroom_name, classroom in classrooms.items():
                            day = random.choice(list(professor['time_available'].keys()))
                            if day in classroom['time_available']:
                                time = random.choice(classroom['time_available'][day])
                                gene = Gene(classgroup_id, classgroup['students'], subject_id, professor_id, day, time, classroom['capacity'])
                                chromosome.add_gene(gene)
        population.add_chromosome(chromosome)
    return population

In [2451]:
def calculate_chromosome_fitness(chromosome: Chromosome) -> int:
    conflicts = 0
    professor_timeslot_usage = {}
    classroom_timeslot_usage = {}

    for gene in chromosome.genes:
        # Verifica conflito de horário do professor
        professor_timeslot_key = (gene.professor_id, gene.professor_day, gene.professor_time)
        if professor_timeslot_key in professor_timeslot_usage:
            conflicts += 1  # Horário duplicado, conflito detectado
        else:
            professor_timeslot_usage[professor_timeslot_key] = 1

        # Verifica conflito de horário da sala
        classroom_timeslot_key = (gene.classroom_capacity, gene.professor_day, gene.professor_time)
        if classroom_timeslot_key in classroom_timeslot_usage:
            conflicts += 1  # Sala duplicada, conflito detectado
        else:
            classroom_timeslot_usage[classroom_timeslot_key] = 1

        # Verifica conflitos de capacidade da sala
        if gene.classgroup_students > gene.classroom_capacity:
            conflicts += 1  # Sala superlotada, conflito detectado

    print(f"Evaluating chromosome with {len(chromosome.genes)} genes, {conflicts} total conflicts")
    return conflicts

In [2452]:
def calculate_chromosome_elite_fitness(chromosome: Chromosome, conflicts: int|None = None) -> float:
    fitness = 1.0
    if conflicts is None:
        conflicts = calculate_chromosome_fitness(chromosome)

    # Reduzir a fitness por conflito
    fitness -= conflicts * 0.1

    if fitness < 0:
        fitness = 0

    return fitness

In [2453]:
def selection(population: Population) -> List[Chromosome]:
    mating_pool = []
    population_fitness = [(chromosome, calculate_chromosome_elite_fitness(chromosome)) for chromosome, _ in population.chromosomes]
    population_fitness.sort(key=lambda x: x[1], reverse=True)

    total_fitness = sum(fitness for _, fitness in population_fitness)
    if total_fitness == 0:
        return [chromosome for chromosome, _ in population_fitness]

    for chromosome, fitness in population_fitness:
        probability = fitness / total_fitness
        if random.random() < probability:
            mating_pool.append(chromosome)

    return mating_pool

In [2454]:
def crossover(mating_pool: List[Chromosome], CROSSOVER_RATE: float) -> List[Chromosome]:
    offspring = []
    num_couples = len(mating_pool) // 2

    for _ in range(num_couples):
        if random.random() < CROSSOVER_RATE:
            parents = random.sample(mating_pool, 2)
            parent1, parent2 = parents[0], parents[1]

            if len(parent1.genes) > 1 and len(parent2.genes) > 1:
                point = random.randint(1, min(len(parent1.genes), len(parent2.genes)) - 1)
                child1 = Chromosome()
                child2 = Chromosome()

                child1.genes = parent1.genes[:point] + parent2.genes[point:]
                child2.genes = parent2.genes[:point] + parent1.genes[point:]

                offspring.append(child1)
                offspring.append(child2)
            else:
                offspring.append(parent1)
                offspring.append(parent2)
    return offspring

In [2455]:
def mutate(mating_pool: List[Chromosome], MUTATION_RATE: float, professors: Dict[str, Professor], subjects: Dict[str, Subject], classrooms: Dict[str, Classroom]) -> List[Chromosome]:
    for chromosome in mating_pool:
        for gene in chromosome.genes:
            if random.random() < MUTATION_RATE:
                professor_id_str = str(gene.professor_id)
                if professor_id_str in professors:
                    available_days = list(professors[professor_id_str]['time_available'].keys())
                    if available_days:
                        new_day = random.choice(available_days)
                        new_time = random.choice(professors[professor_id_str]['time_available'][new_day])
                        gene.professor_day = new_day
                        gene.professor_time = new_time
    return mating_pool

In [2456]:
def replace_population(population: Population, mating_pool: List[Chromosome], calculate_fitness) -> Population:
    new_population = Population()

    mating_pool_fitness = [(chromosome, calculate_fitness(chromosome)) for chromosome in mating_pool]
    mating_pool_fitness.sort(key=lambda x: x[1])  # Ordena por número de conflitos (menor é melhor)

    population_fitness = [(chromosome, calculate_fitness(chromosome)) for chromosome, _ in population.chromosomes]
    population_fitness.sort(key=lambda x: x[1])  # Ordena por número de conflitos (menor é melhor)

    for i in range(len(population_fitness)):
        if i < len(mating_pool_fitness):
            new_population.add_chromosome(mating_pool_fitness[i][0], mating_pool_fitness[i][1])
        else:
            new_population.add_chromosome(population_fitness[i][0], population_fitness[i][1])

    return new_population

# 1 Model

## 1.1 Load Data

In [2457]:
relationship = {
    PROFESSORS_FILE_PATH: Professor,
    SUBJECTS_FILE_PATH: Subject,
    CLASSROOMS_FILE_PATH: Classroom,
    CLASSGROUPS_FILE_PATH: ClassGroup
}

In [2458]:
# professors = load_data(PROFESSORS_FILE_PATH, relationship[PROFESSORS_FILE_PATH])
# professors

professors = [
    {
        "id": "1",
        "name": "Professor A",
        "time_available": {
            "Monday": [9, 10, 11],
            "Wednesday": [9, 10, 11]
        },
        "subjects_taughtble_id": [101, 102]
    },
    {
        "id": "2",
        "name": "Professor B",
        "time_available": {
            "Tuesday": [10, 11],
            "Thursday": [10, 11]
        },
        "subjects_taughtble_id": [103]
    },
    {
        "id": "3",
        "name": "Professor C",
        "time_available": {
            "Monday": [9, 10],
            "Wednesday": [9, 10]
        },
        "subjects_taughtble_id": [101, 104]
    },
    {
        "id": "4",
        "name": "Professor D",
        "time_available": {
            "Monday": [9, 11],
            "Wednesday": [9, 11]
        },
        "subjects_taughtble_id": [102, 104]
    }
]

professors

[{'id': '1',
  'name': 'Professor A',
  'time_available': {'Monday': [9, 10, 11], 'Wednesday': [9, 10, 11]},
  'subjects_taughtble_id': [101, 102]},
 {'id': '2',
  'name': 'Professor B',
  'time_available': {'Tuesday': [10, 11], 'Thursday': [10, 11]},
  'subjects_taughtble_id': [103]},
 {'id': '3',
  'name': 'Professor C',
  'time_available': {'Monday': [9, 10], 'Wednesday': [9, 10]},
  'subjects_taughtble_id': [101, 104]},
 {'id': '4',
  'name': 'Professor D',
  'time_available': {'Monday': [9, 11], 'Wednesday': [9, 11]},
  'subjects_taughtble_id': [102, 104]}]

In [2459]:
# subjects = load_data(SUBJECTS_FILE_PATH, relationship[SUBJECTS_FILE_PATH])
# subjects

subjects = [
    {
        "id": 101,
        "name": "Matemática",
        "professors": ["1", "3"],
        "course_load": 4
    },
    {
        "id": 102,
        "name": "História",
        "professors": ["1", "4"],
        "course_load": 4
    },
    {
        "id": 103,
        "name": "Ciências",
        "professors": ["2"],
        "course_load": 4
    },
    {
        "id": 104,
        "name": "Geografia",
        "professors": ["3", "4"],
        "course_load": 4
    }
]
subjects

[{'id': 101, 'name': 'Matemática', 'professors': ['1', '3'], 'course_load': 4},
 {'id': 102, 'name': 'História', 'professors': ['1', '4'], 'course_load': 4},
 {'id': 103, 'name': 'Ciências', 'professors': ['2'], 'course_load': 4},
 {'id': 104, 'name': 'Geografia', 'professors': ['3', '4'], 'course_load': 4}]

In [2460]:
# classrooms = load_data(CLASSROOMS_FILE_PATH, relationship[CLASSROOMS_FILE_PATH])
# classrooms

classrooms = [
    {
        "id": "Sala 101",
        "name": "Sala 101",
        "time_available": {
            "Monday": [9, 10, 11],
            "Tuesday": [10, 11],
            "Wednesday": [9, 10, 11],
            "Thursday": [10, 11]
        },
        "capacity": 30
    },
    {
        "id": "Sala 102",
        "name": "Sala 102",
        "time_available": {
            "Monday": [9, 10, 11],
            "Wednesday": [9, 10, 11]
        },
        "capacity": 20
    },
    {
        "id": "Sala 103",
        "name": "Sala 103",
        "time_available": {
            "Monday": [9, 11],
            "Wednesday": [9, 11]
        },
        "capacity": 25
    }
]
classrooms

[{'id': 'Sala 101',
  'name': 'Sala 101',
  'time_available': {'Monday': [9, 10, 11],
   'Tuesday': [10, 11],
   'Wednesday': [9, 10, 11],
   'Thursday': [10, 11]},
  'capacity': 30},
 {'id': 'Sala 102',
  'name': 'Sala 102',
  'time_available': {'Monday': [9, 10, 11], 'Wednesday': [9, 10, 11]},
  'capacity': 20},
 {'id': 'Sala 103',
  'name': 'Sala 103',
  'time_available': {'Monday': [9, 11], 'Wednesday': [9, 11]},
  'capacity': 25}]

In [2461]:
# classgroups = load_data(CLASSGROUPS_FILE_PATH, relationship[CLASSGROUPS_FILE_PATH])
# classgroups

classgroups = [
    {"id": 1, "students": 20},
    {"id": 2, "students": 15}
]
classgroups


[{'id': 1, 'students': 20}, {'id': 2, 'students': 15}]

## 1.2 Model Definition

In [2462]:
POPULATION_SIZE = 100
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8
GENERATIONS = 1000
FITNESS_THRESHOLD = 0.9

In [2463]:
# NUM_CLASSES = calculate_quantity_of_classes(subjects, classrooms)

In [2464]:
def print_best_solution(best_chromosome: Chromosome, professors: Dict[str, dict]):
    if best_chromosome is None:
        print("Nenhuma solução encontrada.")
        return

    schedule = {}
    for gene in best_chromosome.genes:
        classgroup = f"ClassGroup {gene.classgroup_id}"
        subject = f"Subject {gene.subject_id}"
        professor = professors[str(gene.professor_id)]["name"]
        day = gene.professor_day
        time = gene.professor_time
        classroom = f"Classroom {gene.classroom_capacity}"

        if day not in schedule:
            schedule[day] = []

        schedule[day].append({
            "Time": time,
            "ClassGroup": classgroup,
            "Subject": subject,
            "Professor": professor,
            "Classroom": classroom
        })

    for day, entries in schedule.items():
        print(f"Day: {day}")
        for entry in sorted(entries, key=lambda x: x["Time"]):
            print(f"  Time: {entry['Time']} | ClassGroup: {entry['ClassGroup']} | Subject: {entry['Subject']} | Professor: {entry['Professor']} | Classroom: {entry['Classroom']}")
        print()


In [2465]:
def genetic_algorithm(population_size: int, mutation_rate: float, crossover_rate: float, generations: int, fitness_threshold: float, subjects: List[Dict[str, Any]], professors: Dict[str, Dict[str, Any]], classrooms: Dict[str, Dict[str, Any]], classgroups: List[Dict[str, Any]]) -> Tuple[Chromosome, MetricsEvaluator]:

    metrics_evaluator = MetricsEvaluator("Genetic Algorithm")
    metrics_evaluator.start_timer()
    population = generate_initial_population(population_size, subjects, professors, classrooms, classgroups)
    best_chromosome = None
    first_generation = True
    last_good_elite_fitness = 0
    last_good_chromosome: Chromosome|None = None
    last_good_generation = 0
    last_good_conflicts = 0

    for generation in range(generations):
        metrics_evaluator.start_iteration_timer()

        if not first_generation:
            mating_pool = selection(population)
            mating_pool = crossover(mating_pool, crossover_rate)
            mating_pool = mutate(mating_pool, mutation_rate, professors, subjects, classrooms)
            population = replace_population(population, mating_pool, calculate_chromosome_fitness)

        population_conflicts = []
        population_elite_fitness = []

        for chromosome, _ in population.chromosomes:
            conflicts = calculate_chromosome_fitness(chromosome)
            population_conflicts.append(conflicts)
            elite_fitness = 1 / (1 + conflicts)  # Ajuste para calcular fitness
            population_elite_fitness.append(elite_fitness)

        conflicts_elite_fitness = min(population_conflicts)
        max_elite_fitness = max(population_elite_fitness)

        if max_elite_fitness > last_good_elite_fitness:
            print(f"New best elite fitness found in generation {generation}")
            last_good_elite_fitness = max_elite_fitness
            last_good_chromosome = population.chromosomes[population_elite_fitness.index(max_elite_fitness)][0]
            last_good_generation = generation
            last_good_conflicts = conflicts_elite_fitness

            if max_elite_fitness >= fitness_threshold:
                print(f"Fitness threshold reached in generation {generation}")
                best_elite_fitness = max_elite_fitness
                best_chromosome = last_good_chromosome
                best_generation = last_good_generation
                best_conflicts = last_good_conflicts

        avg_elite_fitness = sum(population_elite_fitness) / len(population_elite_fitness)
        avg_conflicts = sum(population_conflicts) / len(population_conflicts)
        metrics_evaluator.update_metrics(avg_conflicts, avg_elite_fitness)
        metrics_evaluator.stop_iteration_timer()
        if first_generation:
            first_generation = False

    if best_chromosome is None:
        best_elite_fitness = last_good_elite_fitness
        best_chromosome = last_good_chromosome
        best_generation = last_good_generation
        best_conflicts = last_good_conflicts

    metrics_evaluator.update_best_metric(best_generation, best_chromosome, best_conflicts, best_elite_fitness)

    metrics_evaluator.stop_timer()
    metrics_evaluator.report()
    print(f"Best Generation: {best_generation}, Best chromosome: {best_chromosome}, Conflicts: {best_conflicts}, Elite Fitness: {best_elite_fitness}")
    print_best_solution(best_chromosome, professors)
    return best_chromosome, metrics_evaluator



## 2 Experiment

In [2466]:
subjects_test = [
    {"id": 1, "professors": [1], "course_load": 4},
    {"id": 2, "professors": [2], "course_load": 4},
    {"id": 3, "professors": [3], "course_load": 4},
    {"id": 4, "professors": [4], "course_load": 4},
]

professors_test = {
    "1": {"name": "Professor A", "time_available": {"Monday": [9, 10, 12]}},
    "2": {"name": "Professor B", "time_available": {"Monday": [9]}},
    "3": {"name": "Professor C", "time_available": {"Tuesday": [9, 10], "Thursday": [10]}},
    "4": {"name": "Professor D", "time_available": {"Tuesday": [11, 12], "Thursday": [11, 12]}},
}

classrooms_test = {
    "Sala 101": {"time_available": {"Monday": [9, 10, 11, 12], "Wednesday": [9, 10, 11, 12]}, "capacity": 30},
    "Sala 102": {"time_available": {"Tuesday": [9, 10, 11, 12], "Thursday": [9, 10, 11, 12]}, "capacity": 25},
}

classgroups_test = [
    {"id": 1, "students": 20},
    {"id": 2, "students": 15}
]

best_timetable, metrics = genetic_algorithm(
    50,
    0.1,
    0.7,
    100,
    0.9,
    subjects_test,
    professors_test,
    classrooms_test,
    classgroups_test
)

# best_timetable, metrics = genetic_algorithm(
#     POPULATION_SIZE,
#     MUTATION_RATE,
#     CROSSOVER_RATE,
#     GENERATIONS,
#     FITNESS_THRESHOLD,
#     subjects,
#     professors,
#     classrooms,
#     classgroups
# )

A saída de streaming foi truncada nas últimas 5000 linhas.
Evaluating chromosome with 8 genes, 3 total conflicts
Evaluating chromosome with 8 genes, 3 total conflicts
Evaluating chromosome with 8 genes, 3 total conflicts
Evaluating chromosome with 8 genes, 3 total conflicts
Evaluating chromosome with 8 genes, 3 total conflicts
Evaluating chromosome with 8 genes, 3 total conflicts
Evaluating chromosome with 8 genes, 3 total conflicts
Evaluating chromosome with 8 genes, 3 total conflicts
Evaluating chromosome with 8 genes, 4 total conflicts
Evaluating chromosome with 8 genes, 4 total conflicts
Evaluating chromosome with 8 genes, 4 total conflicts
Evaluating chromosome with 8 genes, 4 total conflicts
Evaluating chromosome with 8 genes, 4 total conflicts
Evaluating chromosome with 8 genes, 4 total conflicts
Evaluating chromosome with 8 genes, 4 total conflicts
Evaluating chromosome with 8 genes, 4 total conflicts
Evaluating chromosome with 8 genes, 4 total conflicts
Evaluating chromosome w

In [2467]:
print(metrics)

MetricsEvaluator of Model: Genetic Algorithm with Metrics: {'time_to_converge': 0.0, 'best_fitness': 1.0, 'best_chromosome': Chromosome(genes=[Gene(classgroup_id=1, classgroup_students=20, subject_id=1, professor_id=1, professor_day='Monday', professor_time=11, classroom_capacity=30), Gene(classgroup_id=2, classgroup_students=15, subject_id=1, professor_id=1, professor_day='Monday', professor_time=12, classroom_capacity=30), Gene(classgroup_id=1, classgroup_students=20, subject_id=2, professor_id=2, professor_day='Monday', professor_time=9, classroom_capacity=30), Gene(classgroup_id=2, classgroup_students=15, subject_id=2, professor_id=2, professor_day='Monday', professor_time=10, classroom_capacity=30), Gene(classgroup_id=1, classgroup_students=20, subject_id=3, professor_id=3, professor_day='Tuesday', professor_time=12, classroom_capacity=25), Gene(classgroup_id=2, classgroup_students=15, subject_id=3, professor_id=3, professor_day='Tuesday', professor_time=10, classroom_capacity=25)

# 3 Final Model

In [2468]:

class MetricsEvaluator:
    def __init__(self, model: str) -> None:
        self.model = model
        self.metrics = {
            "time_to_converge": 0,
            "best_fitness": 0,
            "best_chromosome": None
        }
        self.iteration_times = []


    def __repr__(self) -> str:
        return f"MetricsEvaluator({self.model})"

    def __str__(self) -> str:
        return f"MetricsEvaluator of Model: {self.model} with Metrics: {self.metrics}"

    def start_timer(self):
        self.start_time = time.time()

    def start_iteration_timer(self):
        self.iteration_start_time = time.time()

    def stop_iteration_timer(self):
        iteration_duration = time.time() - self.iteration_start_time
        self.iteration_times.append(iteration_duration)

    def report(self):
        print(f"Total time: {self.total_time} seconds")
        for i, duration in enumerate(self.iteration_times):
            print(f"Iteration {i + 1}: {duration} seconds")

    def stop_timer(self):
        self.metrics["time_to_converge"] = time.time() - self.start_time // 60

    def update_metrics(self, best_chromosome: Chromosome, best_fitness: float):
        self.metrics["best_fitness"] = best_fitness
        self.metrics["best_chromosome"] = best_chromosome

class GeneticAlgorithm:
    def __init__(self, population_size: int, mutation_rate: float, crossover_rate: float, generations: int, fitness_threshold: float, subjects: Dict[str, Subject], professors: Dict[str, Professor], classrooms: Dict[str, Classroom], classgroups: Dict[str, ClassGroup]) -> None:
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.generations = generations
        self.fitness_threshold = fitness_threshold
        self.subjects = subjects
        self.professors = professors
        self.classrooms = classrooms
        self.classrooms = classrooms
        self.classgroups = classgroups


    def generate_initial_population(self) -> Population:
        print(f"Creating population with {self.population_size} chromosomes, {len(self.subjects)} subjects, {len(self.professors)} professors and {len(self.classrooms)} classrooms")
        population = Population()
        for _ in range(self.population_size):
            chromosome = Chromosome()
            for subject, subject_data in self.subjects.items():
                for professor in subject_data.professors:
                    for classgroup in self.classgroups.values():
                        for _ in range((subject_data.course_load // 36)):
                            for classroom, classroom_data in self.classrooms.items():
                                if classgroup.students <= classroom_data.capacity:
                                    day, time = random.choice(list(set(self.professors[professor].availability) & set(classroom_data.availability)))
                                    gene = Gene(self.professors[professor], subject_data, day, time)
                                    chromosome.add_gene(gene)
            population.add_chromosome(chromosome)
        return population

    def calculate_chromosome_fitness(self, chromosome: Chromosome) -> float:
        fitness = 1.0
        conflicts = 0
        timeslot_count = {}
        room_usage = {}
        for gene in chromosome.genes:
            # Contar uso de horários por professor
            if (gene.professor.name, gene.day, gene.time) in timeslot_count:
                timeslot_count[(gene.professor.name, gene.day, gene.time)] += 1
            else:
                timeslot_count[(gene.professor.name, gene.day, gene.time)] = 1
            # Contar uso de salas
            if (gene.subject.name, gene.day, gene.time) in room_usage:
                room_usage[(gene.subject.name, gene.day, gene.time)] += 1
            else:
                room_usage[(gene.subject.name, gene.day, gene.time)] = 1
        for count in timeslot_count.values():
            if count > 1:
                conflicts += (count - 1)
        for count in room_usage.values():
            if count > 1:
                conflicts += (count - 1)
        # Verificar capacidade da sala
        for gene in chromosome.genes:
            if gene.classgroup.students > gene.classroom.capacity:
                conflicts += 1
        # Reduzir a fitness por conflito
        fitness -= conflicts * 0.1
        if fitness < 0:
            fitness = 0
        return fitness

    def selection(self, population: Population) -> List[Chromosome]:
        mating_pool = []
        population_fitness = [(chromosome, self.calculate_chromosome_fitness(chromosome)) for chromosome, _ in population.chromosomes]
        population_fitness.sort(key=lambda x: x[1], reverse=True)
        for i in range(len(population_fitness)):
            if random.random() < (i / len(population_fitness)):
                mating_pool.append(population_fitness[i][0])
        return mating_pool

    def crossover(self, mating_pool: List[Chromosome]) -> List[Chromosome]:
        offspring = []
        for _ in range(len(mating_pool) // 2):
            parent1 = random.choice(mating_pool)
            parent2 = random.choice(mating_pool)
            if random.random() < self.crossover_rate:
                point = random.randint(1, len(parent1.genes) - 1)
                child1 = Chromosome()
                child2 = Chromosome()
                child1.genes = parent1.genes[:point] + parent2.genes[point:]
                child2.genes = parent2.genes[:point] + parent1.genes[point:]
                offspring.extend([child1, child2])
            else:
                offspring.extend([parent1, parent2])
        return offspring

    def mutate(self, mating_pool: List[Chromosome]) -> List[Chromosome]:
        for chromosome in mating_pool:
            for gene in chromosome.genes:
                if random.random() < self.mutation_rate:
                    # available_times = list(set(self.professors[gene.professor.name].time_available) & set(self.classrooms))
                    available_times = list(set(self.professors[gene.professor.name].availability) & set(self.classrooms[gene.classroom.name].availability))
                    day, time = random.choice(available_times)
                    gene.day = day
                    gene.time = time
        return mating_pool

    def replace_population(self, population: Population, mating_pool: List[Chromosome]) -> List[Chromosome]:
        # population_fitness = [(chromosome, self.calculate_chromosome_fitness(chromosome)) for chromosome in population.chromosomes]
        # population_fitness.sort(key=lambda x: x[1])
        # mating_pool_fitness = [(chromosome, self.calculate_chromosome_fitness(chromosome)) for chromosome in mating_pool]
        # mating_pool_fitness.sort(key=lambda x: x[1], reverse=True)

        population_fitness = [(chromosome, self.calculate_chromosome_fitness(chromosome)) for chromosome, _ in population.chromosomes]
        population_fitness.sort(key=lambda x: x[1])

        mating_pool_fitness = [(chromosome, self.calculate_chromosome_fitness(chromosome)) for chromosome in mating_pool]
        mating_pool_fitness.sort(key=lambda x: x[1], reverse=True)

        for i in range(len(mating_pool_fitness)):
          if mating_pool_fitness[i][1] > population_fitness[i][1]:
              population_fitness[i] = mating_pool_fitness[i]
        # return [chromosome for chromosome, fitness in population_fitness]
        new_population = Population()
        for chromosome, fitness in population_fitness:
            new_population.add_chromosome(chromosome, fitness)

        return new_population

    def run(self) -> Tuple[Chromosome, MetricsEvaluator]:
        metrics_evaluator = MetricsEvaluator("Genetic Algorithm")
        metrics_evaluator.start_timer()
        population = generate_initial_population(self.population_size, self.subjects, self.professors, self.classrooms)
        best_chromosome = None
        for generation in range(0, self.generations):
            print(f"Evaluating generation: {generation}")

            # Calcula a "nota" (fitness) de cada conjunto de horários
            population_fitness = [calculate_chromosome_fitness(chromosome) for chromosome in population.chromosomes]

            # Encontra a melhor "nota" atual
            best_fitness = max(population_fitness)

            if best_fitness >= self.fitness_threshold:
                print(f"Fitness threshold reached in generation {generation}")
                best_chromosome = population.chromosomes[population_fitness.index(best_fitness)]
                break

            mating_pool = selection(population)
            mating_pool = crossover(mating_pool, self.crossover_rate)
            mating_pool = mutate(mating_pool, self.mutation_rate, self.professors, self.subjects, self.classrooms)
            population = replace_population(population, mating_pool)

        if best_chromosome is None:
            best_chromosome = population.chromosomes[population_fitness.index(max(population_fitness))]
        metrics_evaluator.stop_timer()
        metrics_evaluator.update_metrics(best_chromosome, best_fitness)
        print(f"Best chromosome: {best_chromosome}, Fitness: {best_fitness}")
        return best_chromosome, metrics_evaluator